# Python私房手册-通过requests库学python

## 参考文档

- [《urllib3官方文档》](https://urllib3.readthedocs.io/en/latest/reference/index.html#urllib3.filepost.encode_multipart_formdata)

## `requests`结构

`requests`所有文件及说明如下：
- `__init__`：所有导入内容的入口
- `__version__`：统一版本
- `_internal_utils`：内部使用的工具函数
- `adapters`：
- `api`：为客户端提供统一的`api`接口
- `auth`：
- `certs`：
- `compat`：为内部程序提供py2，py3兼容性接口
- `cookies`：
- `exceptions`：自定义异常类
- `help`：帮助文件
- `hooks`：提供`default_hooks`和`dispatch_hook`两个钩子函数
- `models`：主要模块，提供`request`和`response`两个类的定义
- `packages`：
- `sessions`：主要模块
- `status_codes`：各种状态码
- `structures`：专用的数据结构类文件，提供`LookupDict`和`CaseInsensitiveDict`两个类似字典的容器类
- `utils`：通用的工具函数

## `models.py`

### `RequestEncodingMixin(object)`混合类

- `path_url`特性：利用`urllib.parse.urlsplit`解析`url`，引申阅读：[《urlsplit和urlparse的区别》](https://blog.csdn.net/mouday/article/details/85613666)
- `_encode_params`静态方法：利用`urllib.parse.urlencode`解析参数，和`urllib.parse.urlencode`区别见附录代码片段1。
- `_encode_files`静态方法：利用`urllib3.fields.RequestField`和`urllib3.filepost.encode_multipart_formdata`构建`content-type`是`multipart/form-data`格式的`Post`请求。见以下引申阅读及附录代码片段2。
 - [《POST之multipart/form-data请求》](https://www.jianshu.com/p/0023bb7afddb)
 - [《Python Requests库 form-data 上传文件操作》](https://www.cnblogs.com/milesma/p/12023405.html)
 - [《Python requests post 提交form-data表单》](https://blog.csdn.net/Chihwei_Hsu/article/details/81943008)
 - [《深入解析 multipart/form-data》](https://blog.csdn.net/wyn126/article/details/96451357)
 - [《HTTP Content-type 对照表》](http://tools.jb51.net/table/http_content_type)

附录代码片段1：

In [31]:
from urllib.parse import urlencode
from requests.models import RequestEncodingMixin

In [29]:
urlencode({"a": 1, "b": [2, 3]})

'a=1&b=%5B2%2C+3%5D'

In [32]:
RequestEncodingMixin._encode_params({"a": 1, "b": [2, 3]})

'a=1&b=2&b=3'

附录代码片段2：
- 注意：源码中`fn`,`fp`,`ft`,`fh`分别指文件名，文件内容，文件类型和文件头，在文件头里加`boundary`无效，`requests`自动生成。

In [88]:
file = {'file': ('log.txt', open("request_log.txt"), "text/plain", {"Connection": "keep-alive"})}
data = {'data': 42}

body, content_type = RequestEncodingMixin._encode_files(files=file, data=data)

print(body.decode("utf8"))

--28edfeef402bcdbc097b26c35ada4d9a
Content-Disposition: form-data; name="data"

42
--28edfeef402bcdbc097b26c35ada4d9a
Content-Disposition: form-data; name="file"; filename="log.txt"
Content-Type: text/plain
Connection: keep-alive

this is a demo
--28edfeef402bcdbc097b26c35ada4d9a--



### `Request(RequestHooksMixin)`类

## 技巧收集

### 如何处理即可以是单个又可以迭代的参数？

以`hooks.py`的`dispatch_hook`函数为例：
```python
def dispatch_hook(key, hooks, hook_data, **kwargs):
    """Dispatches a hook dictionary on a given piece of data."""
    hooks = hooks or {}
    hooks = hooks.get(key)
    if hooks:
        if hasattr(hooks, '__call__'):
            hooks = [hooks]
        for hook in hooks:
            _hook_data = hook(hook_data, **kwargs)
            if _hook_data is not None:
                hook_data = _hook_data
    return hook_data
```

1. 对`hooks`参数的处理，注意与设置默认值`hooks=None`的区别。
2. `hooks`可以为可调用对象，也可以为数组，此时先判断是否为可调用对象，如果是，则设置为数组，接下来统一循环处理即可。